<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/MicrosoftNNI/SimulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Microsoft NNI on Google Colab / Kaggle

In [1]:
# install nni
! pip install nni

# download ngrok and unzip it
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

  Obtaining dependency information for nni from https://files.pythonhosted.org/packages/0b/ed/7a061494753317e0943be1685a4690d05e2e610e5b19aa2823547b584950/nni-3.0-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for json-tricks>=3.15.5 from https://files.pythonhosted.org/packages/ae/fd/e3edcf827e7f9c17c5ea1a192841dcfb1dd575a7518c25c5cadd921625b1/json_tricks-3.17.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for typeguard<4.1.3,>=3.0.0 from https://files.pythonhosted.org/packages/bb/bd/dc7da80c95c920ee2b575e64901b5962ca4a1271b5f3cf6c27242aa0aafc/typeguard-4.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.7.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 MB 18.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: typing-extensions
  

In [2]:
! ./ngrok authtoken 2ZqSaEGuFWdJVe4YADpDuGtZ1R9_2L3tHZaAHSSavE6N96jo4 # YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# clone the github repo to get the model.py file
! git clone https://github.com/antbartash/australian_rain.git

Cloning into 'australian_rain'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 264 (delta 90), reused 105 (delta 57), pack-reused 101
Receiving objects: 100% (264/264), 8.71 MiB | 17.95 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [4]:
from nni.experiment import Experiment

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
hyperparameter_space = {
    'n_estimators': {'_type': 'randint', '_value': [100, 2000]},
    'learning_rate': {'_type': 'uniform', '_value': [1e-6, 0.5]},
    'depth': {'_type': 'randint', '_value': [1, 10]},
    'l2_leaf_reg': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'random_strength': {'_type': 'uniform', '_value': [0.0, 100.0]}, # CPU only
    'bagging_temperature': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'grow_policy': {'_type': 'choice', '_value': ['SymmetricTree', 'Depthwise']}, # Metis supports only numerical hyperparameters
    'scale_pos_weight': {'_type': 'uniform', '_value': [1, 4]}
}

In [6]:
experiment = Experiment('local')
experiment.config.experiment_name = 'CatBoost_NNI_Metis'
experiment.config.tuner.name = 'MetisTuner'
experiment.config.tuner.class_args = {
    'optimize_mode': 'maximize',
}
experiment.config.experiment_working_directory = '/kaggle/working/'
experiment.config.trial_command = 'python3 /kaggle/working/australian_rain/MicrosoftNNI/model_GPU.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = hyperparameter_space
experiment.config.max_trial_number = 100
experiment.config.trial_concurrency = 1
experiment.config.max_experiment_duration = '1h'

In [7]:
get_ipython().system_raw('./ngrok http 5000 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://19b3-34-133-45-156.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://19b3-34-133-45-156.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
experiment.run(5000, wait_completion=True, debug=False)

[2023-12-29 17:36:14] Creating experiment, Experiment ID: zacoiksy
[2023-12-29 17:36:14] Starting web server...
[2023-12-29 17:36:15] Setting up...
[2023-12-29 17:36:16] Web portal URLs: http://127.0.0.1:5000 http://172.19.2.2:5000


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[Selection.py] Exercise "lm" acquisition function
[Selection.py] Exercise "lc" acquisition function
/opt/conda/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
[Selection.py] Exercise "lm" acquisition function
/opt/conda/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    ht

True

In [10]:
best_trial = sorted(experiment.export_data(), key=lambda x: x.value, reverse=True)[0]
print(best_trial.parameter)

{'n_estimators': 187, 'learning_rate': 0.23596287784079056, 'depth': 8, 'l2_leaf_reg': 95.85548143767377, 'bagging_temperature': 1.4085792398285357, 'scale_pos_weight': 1.1430403583421198}


In [11]:
# zip NNI logs
!zip -r MetisLogs.zip /kaggle/working/zacoiksy

  adding: kaggle/working/zacoiksy/ (stored 0%)
  adding: kaggle/working/zacoiksy/checkpoint/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/local-env/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/G4lNX/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/G4lNX/trial.log (deflated 14%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/G4lNX/trial.stdout (deflated 83%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/G4lNX/trial.stderr (deflated 28%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/J9RVW/ (stored 0%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/J9RVW/trial.log (deflated 14%)
  adding: kaggle/working/zacoiksy/environments/local-env/trials/J9RVW/trial.stdout (deflated 83%)
  adding: kaggle/working/zacoiksy/envi

In [12]:
# download NNI logs
from IPython.display import FileLink
FileLink(r'MetisLogs.zip')

/kaggle/working/MetisLogs.zip